### SAGEMAKER MODEL MONITORING IN DKUBE

In [ ]:
!pip install sagemaker
%store -r

### Constants

### Global Values

In [ ]:
import os
os.environ["AWS_ACCESS_KEY_ID"] = ACCESS_KEY
os.environ["AWS_SECRET_ACCESS_KEY"] = SECRET_KEY
os.environ["AWS_DEFAULT_REGION"] = REGION_NAME

### Libraries

In [ ]:
import boto3
import re
import time
import json
import joblib
import pickle
import tarfile
import sagemaker
from sagemaker.estimator import Estimator
import time
from time import gmtime, strftime
import subprocess

### Deploy the model in sagemaker

In [ ]:
data_capture_prefix = "{}/datacapture".format(PREFIX)
s3_capture_upload_path = "s3://{}/{}".format(BUCKET, data_capture_prefix)          
reports_prefix = "{}/reports".format(PREFIX)
s3_report_path = "s3://{}/{}".format(BUCKET, reports_prefix)
code_prefix = "{}/code".format(PREFIX)
s3_code_preprocessor_uri = "s3://{}/{}/{}".format(BUCKET, code_prefix, "preprocessor.py")
s3_code_postprocessor_uri = "s3://{}/{}/{}".format(BUCKET, code_prefix, "postprocessor.py")

print("Capture path: {}".format(s3_capture_upload_path))
print("Report path: {}".format(s3_report_path))
print("Preproc Code path: {}".format(s3_code_preprocessor_uri))
print("Postproc Code path: {}".format(s3_code_postprocessor_uri))

In [ ]:
bashCommand = "tar -cvpzf model.tar.gz model.joblib inference.py"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

#### Upload trained model.joblib file in sagemaker

In [ ]:
model_file = open("model.joblib", "rb")
s3_key = os.path.join(PREFIX, "model.joblib")
boto3.Session().resource("s3").Bucket(BUCKET).Object(s3_key).upload_fileobj(model_file)

In [ ]:
boto_session = boto3.session.Session(aws_access_key_id=ACCESS_KEY,aws_secret_access_key=SECRET_KEY,region_name=REGION_NAME)
sagemaker_session = sagemaker.Session(boto_session)

s3 = boto_session.resource('s3')
default_bucket = sagemaker_session.default_bucket()
#Upload tar.gz to bucket
model_artifacts = f"s3://{default_bucket}/model.tar.gz"
response = s3.meta.client.upload_file('model.tar.gz', BUCKET, 'model.tar.gz')

In [ ]:
%%writefile inference_script.py
import joblib
import os

def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

In [ ]:
from sagemaker.sklearn.model import SKLearnModel

model = SKLearnModel(
    model_data=f"s3://{default_bucket}/model.tar.gz",
    role=ROLE,
    entry_point='inference_script.py',
    framework_version='0.23-1')

### Deploying...

In [ ]:
from sagemaker.model_monitor import DataCaptureConfig

endpoint_name = "DEMO-insurance-model-monitor-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

In [ ]:
%store endpoint_name